# Tree Methods Exercise Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

## Initializing Environment and Loading Data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_exercise').getOrCreate()

23/08/19 20:52:29 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/19 20:52:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 20:52:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 20:52:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/19 20:52:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
data = spark.read.csv('dog_food.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [5]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [7]:
data.filter('Spoiled = 0').show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2| 8.0|  9|    0.0|
|  4|  8| 9.0|  1|    0.0|
| 10|  8| 8.0|  6|    0.0|
|  8|  6| 9.0|  4|    0.0|
|  7|  2| 7.0|  8|    0.0|
|  3|  3| 9.0|  5|    0.0|
|  4| 10| 8.0|  9|    0.0|
|  4|  7|10.0|  7|    0.0|
|  1|  7| 8.0|  2|    0.0|
| 10|  7| 8.0|  5|    0.0|
| 10|  5| 9.0|  1|    0.0|
|  5|  7|10.0| 10|    0.0|
|  2|  8| 6.0|  9|    0.0|
|  4|  1| 7.0|  5|    0.0|
|  4|  6| 9.0|  7|    0.0|
|  2|  2| 9.0|  8|    0.0|
|  6|  7| 6.0|  9|    0.0|
|  5|  7| 7.0|  2|    0.0|
|  7|  1| 7.0|  5|    0.0|
|  8|  1| 8.0|  3|    0.0|
+---+---+----+---+-------+
only showing top 20 rows



In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [11]:
output_data = assembler.transform(data)

In [12]:
output_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
final_data = output_data.select(['features', 'Spoiled'])

In [14]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(labelCol='Spoiled')

In [15]:
rfc_model = rfc.fit(final_data)

In [16]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0131, 1: 0.0223, 2: 0.9477, 3: 0.0169})

This indicates the feature at index 2 (i.e. C) is most responsible for spoiling the Dog Food